In [ ]:
# Imports
import json
import os
import numpy as np
import time
Xs = []
from random import random, randint, choice, shuffle

In [ ]:
def get_dataset():
    dataset = {
        "train": {},
        "test": {},
        "evaluation": {}
    }
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            beg = dirname.split('/')[-1][:3]
            if beg == "abs":
                continue
            print(os.path.join(dirname, filename))
            with open(os.path.join(dirname, filename)) as json_file:
                data = json.load(json_file)
                key = filename[:-5]
                if beg == "eva":
                    dataset["evaluation"][key] = data
                if beg == 'tes':
                    dataset["test"][key] = data
                if beg == 'tra':
                    dataset["train"][key] = data
    return dataset



class Task:
    '''
        Handle a Task of the ARC Dataset
    '''
    def __init__(self, task_name, task_data):
        self.name = task_name
        self.data = task_data

    def examples(self):
        return map(lambda x: (x['input'], x['output']), self.data['train'])

    def test(self):
        return map(lambda x: x['input'], self.data['test'])

    def get_solutions(self):
        return map(lambda x: x['output'], self.data['test'])

class ARC:
    '''
        Handle the ARC Dataset
    '''
    def __init__(self):
        self.dataset = get_dataset()
        self.keys = {}
        for t_type, tasks in self.dataset.items():
            self.keys[t_type] = list(tasks.keys())

    def tasks(self, task_type = 'all'):
        '''
            Iterate over tasks of task_type type

            Params:
                task_type: 'train', 'test', 'eval', 'all'

            Return:
            Iterator
        '''
        for t_type, tasks in self.dataset.items():
            if task_type != 'all' and task_type != t_type:
                continue
            for t_name, task in tasks.items():
                yield Task(t_name, task)
                
    def get_random_task(self, task_type):
        t_name = choice(self.keys[task_type])
        return Task(t_name, self.dataset[task_type][t_name])
    
    def iterate_random_task(self, task_type):
        while 1:
            yield self.get_random_task(task_type)

In [ ]:
arc = ARC()

In [ ]:
MAX_SIZE = 30
NB_CHANNELS = 12

In [ ]:
def get_one_hot_resized(X):
    h, w = X.shape
    resized = np.ones((MAX_SIZE, MAX_SIZE), dtype=np.uint8) * 10
    pos_y, pos_x = MAX_SIZE // 2 - h // 2, MAX_SIZE // 2 - w // 2
    resized[pos_y : pos_y + h, pos_x : pos_x + w] = X
    return np.eye(NB_CHANNELS)[resized[:]].reshape((MAX_SIZE, MAX_SIZE, NB_CHANNELS))

def get_inputs(ttype):
    Xs = []
    for task in arc.tasks(ttype):
        for X, y in task.examples():
            Xs.append(get_one_hot_resized(np.array(X)))
            Xs.append(get_one_hot_resized(np.array(y)))
        for X in task.test():
            Xs.append(get_one_hot_resized(np.array(X)))
        if ttype not in ['train', 'evaluation']:
            continue
        for y in task.get_solutions():
            Xs.append(get_one_hot_resized(np.array(y)))
    return Xs


In [ ]:
Xs = []
Xs += get_inputs('train')
Xs += get_inputs('evaluation')
print(len(Xs), Xs[0].shape)

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

In [ ]:
input_img = Input(shape=(MAX_SIZE, MAX_SIZE, NB_CHANNELS))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.summary()

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')